In [ ]:
%cd ..

In [ ]:
import os
import time
from PIL import Image
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
%matplotlib inline

tf.debugging.set_log_device_placement(False)

os.environ["TFHUB_DOWNLOAD_PROGRESS"] = "True"

dry_run = False

In [ ]:
import tensorflow as tf 
print(tf.test.is_built_with_cuda()) 
print(tf.config.list_physical_devices('GPU'))

In [ ]:
from utils import plot_multiple_images, plot_single_image


plot_multiple_images(tf.random.normal((2, 16, 32, 32, 3)) * 255, 'Gaussian noise')

In [ ]:
from os import path
from data import CLEAR_IMAGES_DIR, get_dataset

X, y = next(iter(get_dataset(path.join(CLEAR_IMAGES_DIR, 'train')).batch(1)))
# plot_multiple_images([tf.expand_dims(X[0], axis=0), tf.expand_dims(X[1], axis=0)], 'Enhanced vs groud truth HR images', cmap='gray', figsize=(6, 6))
plot_single_image(X[0], 'LR image')
plt.figure()
plot_single_image(y[0], 'HR image')

In [ ]:
from src.models.esrgan import ESRGAN


esrgan = ESRGAN()
esrgan.load()

enhanced_batch = esrgan(X)

plot_single_image(enhanced_batch[0], 'ESRGAN enhanced image', cmap='gray')

In [ ]:
# t = time.time()
# net(tf.ones(inp_shape))

# print('Single forward step for PAN took', time.time() - t) # 3.076345920562744

In [ ]:
from os import path
from data import CLEAR_IMAGES_DIR, LR_IMAGE_SIZE, HR_IMAGE_SIZE
from utils import plot_multiple_images, plot_single_image
from src.models.pan import PAN
from itertools import cycle


input_lr_shape = (None, 128, 128, 3)
input_hr_shape = (None, 256, 256, 3)
scale = 2

cpu = '/CPU:0'
# gpu = '/GPU:0'

device = cpu

max_ckpt_to_keep = 3
checkpoint_dir = f'./model/non_extorted/x{scale}_checkpoints'
checkpoint_name = 'pix-att-net-ckpt'


with tf.device(device):
    test_dataset = cycle(get_dataset(path.join(CLEAR_IMAGES_DIR, 'train')).batch(4))
    
    sr_net = PAN(scale, input_lr_shape, checkpoint_dir)
    sr_net.load()

In [ ]:
import cv2


def resize_batch(X, target_size, inter=cv2.INTER_CUBIC):
    return tf.constant([cv2.resize(x, target_size, interpolation=inter).astype(np.uint8) for x in X.numpy().astype(np.uint8)], dtype=tf.uint8)

In [ ]:
X, y = next(test_dataset)
y = tf.cast(y, tf.uint8)

plot_multiple_images([tf.cast(X, tf.uint8), np.ones(X.shape) * 255], figsize=(6, 6), title=f'Source images')

print('Max possible psnr', round(float(tf.reduce_mean(tf.image.psnr(y, y, 255))), 4))
y_hat = resize_batch(X, HR_IMAGE_SIZE, cv2.INTER_LINEAR)
psnr = round(float(tf.reduce_mean(tf.image.psnr(y, y_hat, 255))), 4)
plot_multiple_images([y_hat, y], figsize=(6, 6), title=f'Bilinear inter PSNR: {psnr}')
# plot_multiple_images([y_hat, y], figsize=(6, 6))
# plt.savefig('./x4/bilinear_inter_results.png')


y_hat = resize_batch(X, HR_IMAGE_SIZE, cv2.INTER_CUBIC)
psnr = round(float(tf.reduce_mean(tf.image.psnr(y, y_hat, 255))), 4)
plot_multiple_images([y_hat, y], figsize=(6, 6), title=f'Bicubic inter PSNR: {psnr}')
# plot_multiple_images([y_hat, y], figsize=(6, 6))
# plt.savefig('./x4/bicubic_inter_results.png')

y_hat = sr_net(X, training=False)
psnr = round(float(tf.reduce_mean(tf.image.psnr(y, y_hat, 255))), 4)
msssim = round(float(tf.reduce_mean(tf.image.ssim_multiscale(y_hat, y, 255))), 4)
plot_multiple_images([y_hat, y], figsize=(6, 6), title=f'Model PSNR: {psnr}. MS-SSIM: {msssim}')
# plot_multiple_images([y_hat, y], figsize=(6, 6))
# plt.savefig('./x4/sr_model_inter_results.png')

In [ ]:
from functools import partial


# metrics_test_model = partial(resize_batch, target_size=HR_IMAGE_SIZE, inter=cv2.INTER_LINEAR)
# metrics_test_model = partial(resize_batch, target_size=HR_IMAGE_SIZE, inter=cv2.INTER_CUBIC)
metrics_test_model = partial(sr_net, training=False)

In [ ]:
mean_batch_psnr = 0.0
mean_batch_ssim = 0.0
mean_batch_msssim = 0.0

with tf.device('/CPU:0'):
    batch_size = 50
    dataset = get_dataset(path.join(CLEAR_IMAGES_DIR, 'test'))
    n_batches = 0
    for batch_x, batch_y in dataset.batch(batch_size):
        batch_y = tf.cast(batch_y, tf.uint8)
        y_hat = metrics_test_model(batch_x)
        mean_batch_psnr += tf.reduce_mean(tf.image.psnr(y_hat, batch_y, 255))
        mean_batch_ssim += tf.reduce_mean(tf.image.ssim(y_hat, batch_y, 255))
        mean_batch_msssim += tf.reduce_mean(tf.image.ssim_multiscale(y_hat, batch_y, 255))
        n_batches += 1

mean_batch_psnr /= n_batches
mean_batch_ssim /= n_batches
mean_batch_msssim /= n_batches
print('PSNR:', round(float(mean_batch_psnr), 4))
print('SSIM:', round(float(mean_batch_ssim), 4))
print('MSSSIM:', round(float(mean_batch_msssim), 4))

In [ ]:
from json import load


with open('./data/img/clear_words_coordinates.json') as fid:
    annot = load(fid)
    
list(annot.keys())[:3]

In [ ]:
def get_true_text(img_name):
    return ' '.join((w_st['word'] for w_st in annot[img_name])) + '\f'

repr(get_true_text('image0'))

In [ ]:
import pytesseract as tesseract


# specify path to the preinstalled binary file of tesseract
tesseract.pytesseract.tesseract_cmd = r'C:\Users\Dmytro_Shvetsov\Desktop\tesseract\tesseract.exe'
print('Tesseract version: ', tesseract.get_tesseract_version())

In [ ]:
from src.data import prepare_example
from metrics import relative_distance
from glob import glob

lr_mean_relative_distance = 0.0
sr_mean_relative_distance = 0.0

with tf.device('/CPU:0'):
    test_img_fps = glob(path.join(CLEAR_IMAGES_DIR, 'test/*'))
    for fp in test_img_fps:
        lr_im, hr_im, fname = prepare_example(tf.convert_to_tensor(fp, dtype=tf.string), ret_img_name=True)

        text_true = get_true_text(fname.numpy().decode())
        plt.imshow(lr_im.numpy().astype(np.uint8))
        text_pred = tesseract.image_to_string(lr_im.numpy().astype(np.uint8))
        rel_dist = relative_distance(text_true, text_pred)
        lr_mean_relative_distance += rel_dist
        
        sr_img = sr_net(tf.expand_dims(lr_im, 0), training=False)[0].numpy().astype(np.uint8)
        text_pred = tesseract.image_to_string(sr_img)
        rel_dist = relative_distance(text_true, text_pred)
        sr_mean_relative_distance += rel_dist

lr_mean_relative_distance /= len(test_img_fps)
sr_mean_relative_distance /= len(test_img_fps)
print('Mean relative distance without sr', float(lr_mean_relative_distance))
print('Mean relative distance with sr', float(sr_mean_relative_distance))

In [ ]:
from src.data import prepare_example
from metrics import relative_distance
from glob import glob


test_img_fps = glob(path.join(CLEAR_IMAGES_DIR, 'test/*'))
fp = test_img_fps[5]
lr_im, hr_im, fname = prepare_example(tf.convert_to_tensor(fp, dtype=tf.string), ret_img_name=True, lr_img_size=(64, 64))
sr_img = tf.clip_by_value(sr_net(tf.expand_dims(lr_im, 0), training=False)[0], 0, 255).numpy().astype(np.uint8)
plt.imshow(lr_im.numpy().astype(np.uint8))
plt.figure()
plt.imshow(sr_img)


In [ ]:
def read_image_tf(fp, target_size):
    image_string = tf.io.read_file(fp)
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image_decoded, tf.float32) 
    return tf.image.resize(image_decoded, target_size, method=tf.image.ResizeMethod.BICUBIC)

def prepare_example_tf(hr_image_fp, ret_img_name=False, lr_img_size=(128, 128), hr_img_size=(256, 256)):
    example = read_image_tf(hr_image_fp, LR_IMAGE_SIZE), read_image_tf(hr_image_fp, HR_IMAGE_SIZE)
    if ret_img_name:
        img_name = path.split(str(hr_image_fp))[1].split('.')[0]
        return example + (tf.constant(img_name),)
    else:    
        return example


fp = test_img_fps[5]
lr_im, hr_im, fname = prepare_example_tf(tf.convert_to_tensor(fp, dtype=tf.string), ret_img_name=True)
sr_img = sr_net(lr_im[None], training=False)[0]
plt.imshow(lr_im.numpy().astype(np.uint8))
plt.figure()
plt.imshow(sr_img)
